# ПРОГНОЗ ОТТОКА ПОЛЬЗОВАТЕЛЕЙ КРЕДИТНЫХ КАРТ

## ЗАДАЧА
Необходимо построить прогнозную модель, которая сможет определить, перестанет ли клиент банка пользоваться кредитной картой.

#### В таблице представлены следующие столбцы:

**CLIENTNUM** - Номер клиента. Уникальный идентификатор клиента, которому принадлежит счет

**Attrition_Flag** - Aктивность клиента — если учетная запись закрыта, то Attrited Customer, иначе Existing Customer

**Customer_Age** - Bозраст клиента в годах

**Gender** - М=мужчина, Ж=женщина

**Dependent_count** - Количество иждивенцев

**Education_Level** - Образование, квалификация владельца счета (пример: средняя школа, выпускник колледжа и т. д.)

**Marital_Status** - Женат, Холост, Разведен, Неизвестно

**Income_Category** - Категория годового дохода владельца счета (< 40 000 долларов США, 40 000-60 000 долларов США, 60 000-80 000 долларов США, 80 000-120 000 долларов США, >

**Card_Category** - Тип карты (Синяя, Серебряная, Золотая, Платиновая)

**Months_on_book** - Период отношений с банком

**Total_Relationship_Count** - Общее количество продукции, хранящейся у клиента

**Months_Inactive_12_mon** - Количество месяцев бездействия за последние 12 месяцев

**Contacts_Count_12_mon** - Количество контактов за последние 12 месяцев

**Credit_Limit** - Кредитный лимит по кредитной карте

**Total_Revolving_Bal** - Общий оборотный баланс на кредитной карте

**Avg_Open_To_Buy** - Открытая кредитная линия для покупки (среднее значение за последние 12 месяцев)

**Total_Amt_Chng_Q4_Q1** - Изменение суммы транзакции (Q4 по сравнению с Q1)

**Total_Trans_Amt** - Общая сумма транзакции (последние 12 месяцев)

**Total_Trans_Ct** - Общее количество транзакций (последние 12 месяцев)

**Total_Ct_Chng_Q4_Q1** - Изменение количества транзакций (Q4 по сравнению с Q1)

**Avg_Utilization_Ratio** - Средний коэффициент использования карт

**МЕТРИКА**

Вычислите оценку F1, также известную как сбалансированная F-оценка или F-мера.

Показатель F1 можно интерпретировать как среднее гармоническое между точностью и отзывом, где показатель F1 достигает своего наилучшего значения при 1, а наихудший показатель при 0. Относительный вклад точности и отзыва в показатель F1 равен.

Формула для оценки F1:

F1 = 2 * (precision * recall) / (precision + recall)

In [1]:
import pandas as pd

In [7]:
train_df = pd.read_csv('train.csv')

In [8]:
train_df.head()

,CLIENTNUM,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,...,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Attrition_Flag
0,715630983,31,F,0,Graduate,Single,Less than $40K,Blue,21,4,...,3,4598.0,0,4598.0,0.439,6317,77,0.833,0.000,Attrited Customer
1,713643858,53,F,1,College,Married,$40K - $60K,Blue,43,4,...,2,1525.0,1411,114.0,0.660,1911,47,0.958,0.925,Existing Customer
2,708334158,46,F,3,Graduate,Divorced,Less than $40K,Blue,34,4,...,4,9863.0,686,9177.0,0.581,3068,58,0.933,0.070,Existing Customer
3,711033408,50,F,2,High School,Married,Less than $40K,Blue,36,5,...,2,9355.0,1626,7729.0,0.701,1230,28,0.867,0.174,Existing Customer
4,711368283,39,M,2,Unknown,Married,$80K - $120K,Blue,27,3,...,2,34516.0,1629,32887.0,1.078,3448,53,1.038,0.047,Existing Customer


In [9]:
train_df.tail()

,CLIENTNUM,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,...,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Attrition_Flag
7590,772596783,31,M,1,Graduate,Unknown,Less than $40K,Gold,21,3,...,2,15261.0,1109,14152.0,0.655,15442,125,0.761,0.073,Existing Customer
7591,820330008,46,M,5,High School,Married,$120K +,Blue,42,1,...,3,32926.0,1540,31386.0,0.862,8231,98,0.556,0.047,Existing Customer
7592,720614658,40,M,2,High School,Divorced,$60K - $80K,Blue,25,6,...,2,2978.0,0,2978.0,0.490,3879,95,0.583,0.000,Existing Customer
7593,709759908,46,M,2,Graduate,Married,$60K - $80K,Blue,36,4,...,2,1939.0,685,1254.0,0.421,4327,84,0.680,0.353,Existing Customer
7594,814489683,34,F,2,Unknown,Single,Less than $40K,Blue,29,3,...,0,5387.0,795,4592.0,0.837,2596,59,1.034,0.148,Existing Customer


In [10]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7595 entries, 0 to 7594
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CLIENTNUM                 7595 non-null   int64  
 1   Customer_Age              7595 non-null   int64  
 2   Gender                    7595 non-null   object 
 3   Dependent_count           7595 non-null   int64  
 4   Education_Level           7595 non-null   object 
 5   Marital_Status            7595 non-null   object 
 6   Income_Category           7595 non-null   object 
 7   Card_Category             7595 non-null   object 
 8   Months_on_book            7595 non-null   int64  
 9   Total_Relationship_Count  7595 non-null   int64  
 10  Months_Inactive_12_mon    7595 non-null   int64  
 11  Contacts_Count_12_mon     7595 non-null   int64  
 12  Credit_Limit              7595 non-null   float64
 13  Total_Revolving_Bal       7595 non-null   int64  
 14  Avg_Open

In [12]:
validation_df = pd.read_csv('validation.csv')
validation_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2532 entries, 0 to 2531
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CLIENTNUM                 2532 non-null   int64  
 1   Customer_Age              2532 non-null   int64  
 2   Gender                    2532 non-null   object 
 3   Dependent_count           2532 non-null   int64  
 4   Education_Level           2532 non-null   object 
 5   Marital_Status            2532 non-null   object 
 6   Income_Category           2532 non-null   object 
 7   Card_Category             2532 non-null   object 
 8   Months_on_book            2532 non-null   int64  
 9   Total_Relationship_Count  2532 non-null   int64  
 10  Months_Inactive_12_mon    2532 non-null   int64  
 11  Contacts_Count_12_mon     2532 non-null   int64  
 12  Credit_Limit              2532 non-null   float64
 13  Total_Revolving_Bal       2532 non-null   int64  
 14  Avg_Open

In [14]:
train_df.Attrition_Flag.value_counts()

Existing Customer    6375
Attrited Customer    1220
Name: Attrition_Flag, dtype: int64